In [1]:
from py2neo import Graph
import numpy as np
import pandas as pd
import os

In [2]:
class Py2NeoGraph(object):
    
    def __init__(self, user:str, password:str):
        self._graph = Graph('http://neo4j:7474/', user=user, password=password)
        self._exports = ['export.auth-citations',
                            'export.authors-of-article',
                            'export.authors-of-work',
                            'export.co-authors',
                            'export.journal-articles',
                            'export.organizations']
    
    @property
    def graph(self):
        return self._graph
    
    @graph.setter
    def graph(self, graph: Graph):
        self._graph = graph
        
    @property
    def exports(self):
        return self._exports
    
    #### TODO: make this better
    def load_all_schemas_cypher(self):
        tx = self.graph.begin()
        for export in self.exports:
            tx.run('CALL apoc.cypher.runSchemaFile("/var/lib/neo4j/import/schema/%s.schema.cypher")' % export)
        tx.commit()
    
    #### TODO: make this better
    def load_all_data_cypher(self):
        tx = self.graph.begin()
        for export in self.exports:
            tx.run('''
                    CALL apoc.cypher.runFiles(["/var/lib/neo4j/import/load/%s.nodes.cypher",
                   "/var/lib/neo4j/import/load/%s.relationships.cypher"])'''
                    % (export, export))
        tx.commit()
    
    #### TODO: make this better
    def clear_all_data_cypher(self):
        tx = self.graph.begin()
        tx.run("CALL apoc.periodic.iterate('MATCH (n) RETURN n', 'DETACH DELETE n', {batchSize:50})")
        tx.commit()
        
#### If batch loading, using tx.process() every 1000 or so

In [3]:
graph = Py2NeoGraph(user='', password='')

In [4]:
graph.load_all_schemas_cypher()

In [5]:
graph.load_all_data_cypher()

In [8]:
!pip install ampligraph

In [9]:
import ampligraph

In [10]:
ampligraph.__version__

'1.3.2'